<a href="https://colab.research.google.com/github/scj02/GitTutorial/blob/master/Prueba_Gemelo.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
{
 "cells": [
  {
   "cell_type": "markdown",
   "id": "18faf328",
   "metadata": {},
   "source": [
    "\n",
    "# Prototipo de Gemelo Digital + Blockchain (Fases 1 a 8)\n",
    "\n",
    "Este notebook integra todas las fases del modelo híbrido:\n",
    "\n",
    "- **Fases 1 a 3:** Gemelo Digital básico con atributos, reglas y validación.\n",
    "- **Fase 4:** Generación de hash (simulación de anclaje en blockchain).\n",
    "- **Fase 5:** Firma digital con ECDSA + ejemplo de anclaje en testnet (Web3.py).\n",
    "- **Fase 6 (Opcional):** Tokenización como NFT (no incluida aquí para mantener claridad).\n",
    "- **Fase 7:** Persistencia en CSV y recarga de diplomas.\n",
    "- **Fase 8:** Verificación de integridad y visualización del ciclo de vida.\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "638d028e",
   "metadata": {},
   "outputs": [],
   "source": [
    "\n",
    "import hashlib\n",
    "import json\n",
    "import time\n",
    "from datetime import datetime\n",
    "\n",
    "import pandas as pd\n",
    "import matplotlib.pyplot as plt\n",
    "\n",
    "from cryptography.hazmat.primitives import hashes\n",
    "from cryptography.hazmat.primitives.asymmetric import ec\n",
    "from cryptography.hazmat.primitives import serialization\n",
    "from cryptography.exceptions import InvalidSignature\n",
    "\n",
    "# Web3 (requiere conexión a un proveedor como Infura o Alchemy en testnet)\n",
    "from web3 import Web3\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "e20bd2b8",
   "metadata": {},
   "outputs": [],
   "source": [
    "\n",
    "# Gemelo Digital inicial\n",
    "diploma = {\n",
    "    \"id\": \"DIP-001\",\n",
    "    \"estudiante\": \"Juan Pérez\",\n",
    "    \"programa\": \"Ingeniería de Sistemas\",\n",
    "    \"estado\": \"En revisión\",\n",
    "    \"atributos\": {\n",
    "        \"paz_y_salvo\": True,\n",
    "        \"notas_completas\": True,\n",
    "        \"deudas\": False,\n",
    "        \"creditos_cumplidos\": 160,\n",
    "        \"creditos_totales\": 160\n",
    "    },\n",
    "    \"historial\": []\n",
    "}\n",
    "\n",
    "print(\"Gemelo Digital inicial:\", json.dumps(diploma, indent=2, ensure_ascii=False))\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "3d5ca39f",
   "metadata": {},
   "outputs": [],
   "source": [
    "\n",
    "def actualizar_estado(diploma, nuevo_estado):\n",
    "    diploma[\"estado\"] = nuevo_estado\n",
    "    diploma[\"historial\"].append({\n",
    "        \"estado\": nuevo_estado,\n",
    "        \"timestamp\": datetime.utcnow().isoformat() + \"Z\"\n",
    "    })\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "fd65c5ff",
   "metadata": {},
   "outputs": [],
   "source": [
    "\n",
    "def validar_diploma(d):\n",
    "    reglas = []\n",
    "    if not d[\"atributos\"][\"paz_y_salvo\"]:\n",
    "        reglas.append(\"🚩 No está paz y salvo\")\n",
    "    if d[\"atributos\"][\"deudas\"]:\n",
    "        reglas.append(\"🚩 Tiene deudas pendientes\")\n",
    "    if not d[\"atributos\"][\"notas_completas\"]:\n",
    "        reglas.append(\"🚩 No completó todas las asignaturas\")\n",
    "    if d[\"atributos\"][\"creditos_cumplidos\"] < d[\"atributos\"][\"creditos_totales\"]:\n",
    "        reglas.append(\"🚩 No ha completado todos los créditos\")\n",
    "    return reglas\n",
    "\n",
    "banderas = validar_diploma(diploma)\n",
    "if banderas:\n",
    "    actualizar_estado(diploma, \"Bloqueado - Banderas rojas\")\n",
    "else:\n",
    "    actualizar_estado(diploma, \"Apto para emisión\")\n",
    "\n",
    "print(\"Validaciones:\", banderas if banderas else \"✅ Todo en orden\")\n",
    "print(\"Estado actual:\", diploma[\"estado\"])\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "07a16446",
   "metadata": {},
   "outputs": [],
   "source": [
    "\n",
    "# Serialización determinista y hash\n",
    "diploma_json = json.dumps(diploma, sort_keys=True, ensure_ascii=False).encode()\n",
    "hash_diploma = hashlib.sha256(diploma_json).hexdigest()\n",
    "\n",
    "diploma[\"hash\"] = hash_diploma\n",
    "diploma[\"historial\"].append({\n",
    "    \"evento\": \"Hash generado\",\n",
    "    \"hash\": hash_diploma,\n",
    "    \"timestamp\": datetime.utcnow().isoformat() + \"Z\"\n",
    "})\n",
    "\n",
    "print(\"Gemelo Digital actualizado:\", json.dumps(diploma, indent=2, ensure_ascii=False))\n",
    "print(\"Hash anclado (simulado blockchain):\", hash_diploma)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "1772178f",
   "metadata": {},
   "outputs": [],
   "source": [
    "\n",
    "# Generación de claves ECDSA (p256)\n",
    "private_key = ec.generate_private_key(ec.SECP256R1())\n",
    "public_key = private_key.public_key()\n",
    "\n",
    "# Firmar hash\n",
    "hash_bytes = bytes.fromhex(hash_diploma)\n",
    "signature = private_key.sign(hash_bytes, ec.ECDSA(hashes.SHA256()))\n",
    "\n",
    "# Verificar firma\n",
    "try:\n",
    "    public_key.verify(signature, hash_bytes, ec.ECDSA(hashes.SHA256()))\n",
    "    print(\"✅ Firma verificada correctamente\")\n",
    "except InvalidSignature:\n",
    "    print(\"❌ Firma inválida\")\n",
    "\n",
    "# Exportar clave pública PEM\n",
    "pem_pub = public_key.public_bytes(\n",
    "    encoding=serialization.Encoding.PEM,\n",
    "    format=serialization.PublicFormat.SubjectPublicKeyInfo\n",
    ")\n",
    "print(pem_pub.decode())\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "4a03d19f",
   "metadata": {},
   "outputs": [],
   "source": [
    "\n",
    "# Ejemplo de anclaje en testnet con Web3.py (requiere Infura/Alchemy y clave privada)\n",
    "\n",
    "# w3 = Web3(Web3.HTTPProvider(\"https://sepolia.infura.io/v3/YOUR_PROJECT_ID\"))\n",
    "# acct = w3.eth.account.from_key(\"YOUR_PRIVATE_KEY\")\n",
    "# tx = {\n",
    "#     'to': acct.address,\n",
    "#     'value': 0,\n",
    "#     'data': bytes.fromhex(hash_diploma),\n",
    "#     'gas': 21000,\n",
    "#     'gasPrice': w3.toWei('10', 'gwei'),\n",
    "#     'nonce': w3.eth.get_transaction_count(acct.address)\n",
    "# }\n",
    "# signed = acct.sign_transaction(tx)\n",
    "# tx_hash = w3.eth.send_raw_transaction(signed.rawTransaction)\n",
    "# print(\"Transacción enviada, hash:\", tx_hash.hex())\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "60017b8a",
   "metadata": {},
   "outputs": [],
   "source": [
    "\n",
    "df = pd.DataFrame([{\n",
    "    \"id\": diploma[\"id\"],\n",
    "    \"estudiante\": diploma[\"estudiante\"],\n",
    "    \"programa\": diploma[\"programa\"],\n",
    "    \"estado\": diploma[\"estado\"],\n",
    "    \"hash\": diploma[\"hash\"],\n",
    "    \"historial\": json.dumps(diploma[\"historial\"], ensure_ascii=False)\n",
    "}])\n",
    "df.to_csv(\"diplomas.csv\", index=False, encoding=\"utf-8\")\n",
    "print(\"Diploma guardado en diplomas.csv\")\n",
    "\n",
    "df_loaded = pd.read_csv(\"diplomas.csv\", encoding=\"utf-8\")\n",
    "print(\"Diplomas cargados:\")\n",
    "print(df_loaded)\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "b9097d1c",
   "metadata": {},
   "outputs": [],
   "source": [
    "\n",
    "diploma_loaded = diploma.copy()\n",
    "diploma_loaded_json = json.dumps(diploma_loaded, sort_keys=True, ensure_ascii=False).encode()\n",
    "hash_recalc = hashlib.sha256(diploma_loaded_json).hexdigest()\n",
    "\n",
    "print(\"Hash guardado:\", diploma[\"hash\"])\n",
    "print(\"Hash recalculado:\", hash_recalc)\n",
    "print(\"✅ Integridad OK\" if diploma[\"hash\"] == hash_recalc else \"❌ Integridad comprometida\")\n"
   ]
  },
  {
   "cell_type": "code",
   "execution_count": null,
   "id": "19e72892",
   "metadata": {},
   "outputs": [],
   "source": [
    "\n",
    "historial_df = pd.DataFrame(diploma[\"historial\"])\n",
    "historial_df[\"timestamp\"] = pd.to_datetime(historial_df[\"timestamp\"])\n",
    "\n",
    "plt.figure(figsize=(8,4))\n",
    "plt.plot(historial_df[\"timestamp\"], historial_df.index, marker=\"o\")\n",
    "for i, row in historial_df.iterrows():\n",
    "    plt.text(row[\"timestamp\"], i, row[\"estado\"], fontsize=9, va=\"bottom\")\n",
    "plt.title(\"Ciclo de vida del diploma\")\n",
    "plt.xlabel(\"Tiempo\")\n",
    "plt.ylabel(\"Secuencia de estados\")\n",
    "plt.grid(True)\n",
    "plt.show()\n"
   ]
  }
 ],
 "metadata": {},
 "nbformat": 4,
 "nbformat_minor": 5
}


NameError: name 'null' is not defined

In [3]:
!pip install web3

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 17.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 587.5/587.5 kB 39.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 102.5/102.5 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.4/51.4 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 332.9/332.9 kB 27.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.4/171.4 kB 14.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 53.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.4/48.4 kB 3.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 55.5 MB/s eta 0:00:00


In [4]:
import hashlib
import json
import time
from datetime import datetime

import pandas as pd
import matplotlib.pyplot as plt

from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.primitives.asymmetric import ec
from cryptography.hazmat.primitives import serialization
from cryptography.exceptions import InvalidSignature

# Web3 (requiere conexión a un proveedor como Infura o Alchemy en testnet)
from web3 import Web3

In [5]:
# Gemelo Digital inicial
diploma = {
    "id": "DIP-001",
    "estudiante": "Juan Pérez",
    "programa": "Ingeniería de Sistemas",
    "estado": "En revisión",
    "atributos": {
        "paz_y_salvo": True,
        "notas_completas": True,
        "deudas": False,
        "creditos_cumplidos": 160,
        "creditos_totales": 160
    },
    "historial": []
}

print("Gemelo Digital inicial:", json.dumps(diploma, indent=2, ensure_ascii=False))

Gemelo Digital inicial: {
  "id": "DIP-001",
  "estudiante": "Juan Pérez",
  "programa": "Ingeniería de Sistemas",
  "estado": "En revisión",
  "atributos": {
    "paz_y_salvo": true,
    "notas_completas": true,
    "deudas": false,
    "creditos_cumplidos": 160,
    "creditos_totales": 160
  },
  "historial": []
}


In [6]:
def actualizar_estado(diploma, nuevo_estado):
    diploma["estado"] = nuevo_estado
    diploma["historial"].append({
        "estado": nuevo_estado,
        "timestamp": datetime.utcnow().isoformat() + "Z"
    })

In [7]:
def validar_diploma(d):
    reglas = []
    if not d["atributos"]["paz_y_salvo"]:
        reglas.append("🚩 No está paz y salvo")
    if d["atributos"]["deudas"]:
        reglas.append("🚩 Tiene deudas pendientes")
    if not d["atributos"]["notas_completas"]:
        reglas.append("🚩 No completó todas las asignaturas")
    if d["atributos"]["creditos_cumplidos"] < d["atributos"]["creditos_totales"]:
        reglas.append("🚩 No ha completado todos los créditos")
    return reglas

banderas = validar_diploma(diploma)
if banderas:
    actualizar_estado(diploma, "Bloqueado - Banderas rojas")
else:
    actualizar_estado(diploma, "Apto para emisión")

print("Validaciones:", banderas if banderas else "✅ Todo en orden")
print("Estado actual:", diploma["estado"])

Validaciones: ✅ Todo en orden
Estado actual: Apto para emisión


/tmp/ipython-input-4042965120.py:5: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat() + "Z"


In [8]:
# Serialización determinista y hash
diploma_json = json.dumps(diploma, sort_keys=True, ensure_ascii=False).encode()
hash_diploma = hashlib.sha256(diploma_json).hexdigest()

diploma["hash"] = hash_diploma
diploma["historial"].append({
    "evento": "Hash generado",
    "hash": hash_diploma,
    "timestamp": datetime.utcnow().isoformat() + "Z"
})

print("Gemelo Digital actualizado:", json.dumps(diploma, indent=2, ensure_ascii=False))
print("Hash anclado (simulado blockchain):", hash_diploma)

Gemelo Digital actualizado: {
  "id": "DIP-001",
  "estudiante": "Juan Pérez",
  "programa": "Ingeniería de Sistemas",
  "estado": "Apto para emisión",
  "atributos": {
    "paz_y_salvo": true,
    "notas_completas": true,
    "deudas": false,
    "creditos_cumplidos": 160,
    "creditos_totales": 160
  },
  "historial": [
    {
      "estado": "Apto para emisión",
      "timestamp": "2025-10-03T16:29:49.568553Z"
    },
    {
      "evento": "Hash generado",
      "hash": "b28b4b5f63c64d70f70073874149dbca7d3193560f61c2f4dc353ab34a4f181d",
      "timestamp": "2025-10-03T16:30:34.768469Z"
    }
  ],
  "hash": "b28b4b5f63c64d70f70073874149dbca7d3193560f61c2f4dc353ab34a4f181d"
}
Hash anclado (simulado blockchain): b28b4b5f63c64d70f70073874149dbca7d3193560f61c2f4dc353ab34a4f181d


/tmp/ipython-input-2169702725.py:9: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  "timestamp": datetime.utcnow().isoformat() + "Z"


In [9]:
# Generación de claves ECDSA (p256)
private_key = ec.generate_private_key(ec.SECP256R1())
public_key = private_key.public_key()

# Firmar hash
hash_bytes = bytes.fromhex(hash_diploma)
signature = private_key.sign(hash_bytes, ec.ECDSA(hashes.SHA256()))

# Verificar firma
try:
    public_key.verify(signature, hash_bytes, ec.ECDSA(hashes.SHA256()))
    print("✅ Firma verificada correctamente")
except InvalidSignature:
    print("❌ Firma inválida")

# Exportar clave pública PEM
pem_pub = public_key.public_bytes(
    encoding=serialization.Encoding.PEM,
    format=serialization.PublicFormat.SubjectPublicKeyInfo
)
print(pem_pub.decode())

✅ Firma verificada correctamente
-----BEGIN PUBLIC KEY-----
MFkwEwYHKoZIzj0CAQYIKoZIzj0DAQcDQgAECHQIznNAnjoVNY6q1q1SJmP8NIzj
ngs2Y9cc1xi+8T+tpOWuMuy3yRFgq23Ndi7hnu3gSsgLtbvl1VgFmgSKPg==
-----END PUBLIC KEY-----



In [12]:
# Ejemplo de anclaje en testnet con Web3.py (requiere Infura/Alchemy y clave privada)

w3 = Web3(Web3.HTTPProvider("https://mainnet.infura.io/v3/1e0a83b4b7ea4916ba64da9e4d1213a9"))
acct = w3.eth.account.from_key("nPyFqucODyLVpBboXlTTOOpKSdCmTggRisoN76EtiXlXcSK9hUDf2g")
tx = {
    'to': acct.address,
    'value': 0,
    'data': bytes.fromhex(hash_diploma),
    'gas': 21000,
    'gasPrice': w3.toWei('10', 'gwei'),
    'nonce': w3.eth.get_transaction_count(acct.address)
}
signed = acct.sign_transaction(tx)
tx_hash = w3.eth.send_raw_transaction(signed.rawTransaction)
print("Transacción enviada, hash:", tx_hash.hex())

Error: Non-hexadecimal digit found

In [13]:
df = pd.DataFrame([{
    "id": diploma["id"],
    "estudiante": diploma["estudiante"],
    "programa": diploma["programa"],
    "estado": diploma["estado"],
    "hash": diploma["hash"],
    "historial": json.dumps(diploma["historial"], ensure_ascii=False)
}])
df.to_csv("diplomas.csv", index=False, encoding="utf-8")
print("Diploma guardado en diplomas.csv")

df_loaded = pd.read_csv("diplomas.csv", encoding="utf-8")
print("Diplomas cargados:")
print(df_loaded)

Diploma guardado en diplomas.csv
Diplomas cargados:
        id  estudiante                programa             estado  \
0  DIP-001  Juan Pérez  Ingeniería de Sistemas  Apto para emisión   

                                                hash  \
0  b28b4b5f63c64d70f70073874149dbca7d3193560f61c2...   

                                           historial  
0  [{"estado": "Apto para emisión", "timestamp": ...  


In [ ]:
diploma_loaded = diploma.copy()
diploma_loaded_json = json.dumps(diploma_loaded, sort_keys=True, ensure_ascii=False).encode()
hash_recalc = hashlib.sha256(diploma_loaded_json).hexdigest()

print("Hash guardado:", diploma["hash"])
print("Hash recalculado:", hash_recalc)
print("✅ Integridad OK" if diploma["hash"] == hash_recalc else "❌ Integridad comprometida")

In [ ]:
historial_df = pd.DataFrame(diploma["historial"])
historial_df["timestamp"] = pd.to_datetime(historial_df["timestamp"])

plt.figure(figsize=(8,4))
plt.plot(historial_df["timestamp"], historial_df.index, marker="o")
for i, row in historial_df.iterrows():
    plt.text(row["timestamp"], i, row["estado"], fontsize=9, va="bottom")
plt.title("Ciclo de vida del diploma")
plt.xlabel("Tiempo")
plt.ylabel("Secuencia de estados")
plt.grid(True)
plt.show()